In [ ]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import rtree

from scipy.spatial import distance
from scipy.spatial.distance import cdist
from scipy import stats
import libpysal as ps 
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap

import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
from matplotlib.colors import ListedColormap
import seaborn as sns


from scipy.stats import jarque_bera, kstest
from statsmodels.compat import lzip
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA

plt.rcParams.update({'font.size': 16})

# Data Loading

In [ ]:
### Load dataset

social = pd.read_csv('https://raw.githubusercontent.com/nampuero/capstone/main/social_infrastructure/social.csv')

In [ ]:
### Remove Staten Island observations and transform skewed distributions

social = social[~social['geolabel'].str.contains('Richmond')]

p_root = 3
p = social['population'] ** (1/p_root)
q = p.quantile([0.25, 0.75]).values
o = 1.5 * float(np.diff(q))
p_out = [q[0] - o, q[1] + o]

w_root = 1
w = social['epa_walk'] ** (1/w_root)
q = w.quantile([0.25, 0.75]).values
o = 1.5 * float(np.diff(q))
w_out = [q[0] - o, q[1] + o]

fig, ax = plt.subplots(2, 1, figsize=(7, 10))

ax[0].hist(p, bins=100)
for out in p_out:
  ax[0].axvline(out, color='tab:red')
ax[0].set_title(f'Population^{p_root} and "whiskers"')

ax[1].hist(w, bins=100)
for out in w_out:
  ax[1].axvline(out, color='tab:red')
ax[1].set_title('Walkability and "whiskers"')

plt.show()

In [ ]:
### Filter outlier observations for population and walkability

social = social.astype(float, errors='ignore')
social = social[(social['population'] > 1000) & (social['epa_walk'] > 10)]

social['cfb_votes'] = social['cfb_votes'] / social['population'] * 100

infra_cols = ['nc_agriculture', 'pluto_church', 'pluto_cultural', 'pluto_outdoor', 'pluto_school']
social[infra_cols] = social[infra_cols].apply(lambda x: x / social['population'])


log_cols = ['acs_bachelors', 'acs_income']
cols = [f'{col}_log' for col in log_cols]
social[log_cols] = social[log_cols].astype(float)
social[cols] = social[log_cols].apply(lambda x: np.log(x+1))

social['noise'] = np.random.randn(len(social))

social.drop(columns=['geolabel', 'population', 'area', *log_cols], inplace=True)
len(social)

In [ ]:
boro_counties_name = {'Manhattan': '061', 'Bronx': '005', 'Brooklyn': '047', 'Queens': '081', 'Staten Island': '085'}

In [ ]:
### merge variables with shapefile for visualization

tract2 = gpd.read_file('nyct2010_21c/nyct2010.shp').to_crs(epsg=4269)
tract2 = tract2[~(tract2['BoroCode'] == '5')]
countycode_dict = {'1':'061', '2':'005', '3':'047', '4':'081'}
tract2['BoroCode'] = tract2['BoroCode'].map(countycode_dict)
tract2['geoid'] = ('36' + tract2['BoroCode'].str.zfill(3) + tract2['CT2010'].str.zfill(6)).astype(int)

In [ ]:
social_geo = gpd.GeoDataFrame(social.merge(tract2[['geoid','geometry']]))

# Feature Engineering

In [ ]:
import scipy
import scipy.cluster.hierarchy as sch

def cluster_corr(corr_array, inplace=False):
    """
    Rearranges the correlation matrix, corr_array, so that groups of highly 
    correlated variables are next to eachother 
    
    Parameters
    ----------
    corr_array : pandas.DataFrame or numpy.ndarray
        a NxN correlation matrix 
        
    Returns
    -------
    pandas.DataFrame or numpy.ndarray
        a NxN correlation matrix with the columns and rows rearranged
    """
    pairwise_distances = sch.distance.pdist(corr_array)
    linkage = sch.linkage(pairwise_distances, method='complete')
    cluster_distance_threshold = pairwise_distances.max()/2
    idx_to_cluster_array = sch.fcluster(linkage, cluster_distance_threshold, 
                                        criterion='distance')
    idx = np.argsort(idx_to_cluster_array)
    
    if not inplace:
        corr_array = corr_array.copy()
    
    if isinstance(corr_array, pd.DataFrame):
        return corr_array.iloc[idx, :].T.iloc[idx, :]
    return corr_array[idx, :][:, idx]

In [ ]:
fig, ax = plt.subplots(figsize=(14,10))
sns.heatmap(cluster_corr(social.corr()), cmap='seismic')

In [ ]:
abs(social.drop(columns=['density','geoid']).corr())['cdc_mental'].sort_values()

### Transform three most correllated and multicollinear features into single PCA

In [ ]:
X1 = social[['acs_bachelors_log', 'acs_income_log', 'cdc_physical']]

sc = StandardScaler()
X1_sc = sc.fit_transform(X1)

In [ ]:
#perform PC decomposition
pca = PCA(X1_sc.shape[1])
pca1 = pca.fit_transform(X1_sc)
eigenvalues = pca.explained_variance_ratio_
#plot explained variance over the number of compinents
n=X1_sc.shape[1]
plt.bar(np.arange(n), eigenvalues[:n].cumsum())
plt.xlabel("Number of components")
plt.ylabel("Explained Variance")
plt.show()

In [ ]:
#perform PC decomposition over data311
pca = PCA(1)
PCA1 = pca.fit_transform(X1_sc)


In [ ]:
social['econ_ed_phys_PCA'] = PCA1

# Define Feature Lists

In [ ]:
features = sorted(['cdc_physical', 'acs_participation', 'acs_uninsured', 'acs_white', 'acs_foreign', 'acs_child',
       'acs_commute60', 'epa_walk', 'cfb_votes', 'nypd_violent', 'nc_agriculture',  'pluto_vacant', 
        'pluto_church', 'pluto_school', 'pluto_cultural', 'pluto_outdoor', 'acs_bachelors_log',
        'acs_income_log', 'noise', 'census_response'])

In [ ]:
pca_features = sorted(['acs_participation', 'acs_uninsured', 'acs_white', 'acs_foreign', 'acs_child',
       'acs_commute60', 'epa_walk', 'cfb_votes', 'nypd_violent', 'nc_agriculture', 'pluto_vacant', 
        'pluto_church', 'pluto_school', 'pluto_cultural', 'pluto_outdoor', 
                           'econ_ed_phys_PCA', 'noise', 'census_response'])


# Geographic Weighted Regression

In [ ]:
### Import Census Tract Centorid Point Locations for GWR calibration

In [ ]:
centroids = gpd.read_file('popctr_tracts2010/popctr_tracts2010.shp')
centroids.columns = centroids.columns.str.lower()
centroids['geoid'] = ('36' + centroids['county'].str.zfill(3) + centroids['tract'].str.zfill(6)).astype(int)
centroids['geoid'] = centroids['geoid'].astype(int)
centroids = centroids[centroids['county'].isin(['061','005','047','081','085'])]
centroids = centroids[centroids['state'] == '36']

In [ ]:
### Merge variables dataframe with centroid geometries on 'geoid'

social_gwr = gpd.GeoDataFrame(social.merge(centroids))

In [ ]:
coordinates = np.column_stack([social_gwr.to_crs('EPSG:4326').longitude,
                                  social_gwr.to_crs('EPSG:4326').latitude])

In [ ]:
# Define target and independent variables

X = social[pca_features]
y = social['cdc_mental']
y = np.array(y).reshape((-1,1))

# Standardize features and target variable for regression

sc = StandardScaler()
X_sc = sc.fit_transform(X)
y_sc = sc.fit_transform(y)

In [ ]:
gwr_selector = Sel_BW(coordinates, y_sc, X_sc, spherical=True)
gwr_bw = gwr_selector.search(bw_min=2)
print(gwr_bw)

In [ ]:
gwr_results = GWR(coordinates, y, X_sc, gwr_bw, spherical=True).fit()

In [ ]:
gwr_results.summary()

In [ ]:
#Prepare GWR results for mapping

gwr_viz = social_geo[['geoid','cdc_mental','geometry']].copy()


#Add GWR parameters to GeoDataframe

for i,j in enumerate(pca_features):
    gwr_viz['gwr_intercept'] = gwr_results.params[:,0]
    gwr_viz['coef_'+j] = gwr_results.params[:,(i+1)]

gwr_viz['R2'] = gwr_results.localR2
gwr_viz['residual'] = gwr_results.std_res

#Obtain t-vals filtered based on multiple testing correction
gwr_filtered_t = gwr_results.filter_tvals()

In [ ]:
gwr_viz.head()

In [ ]:
# Plot estimated global GWR coefficients

coefficients = {
    'feat': pca_features,
    'coef': gwr_results.params.mean(axis=0)[1:],
    'min' : gwr_results.params.min(axis=0)[1:],
    'max' : gwr_results.params.max(axis=0)[1:],
    'abs': np.abs(gwr_results.params.mean(axis=0)[1:]),
    'std': gwr_results.params.std(axis=0)[1:]
}
coefficients = pd.DataFrame(coefficients).sort_values('abs')
noise_coef = coefficients.loc[coefficients['feat'] == 'noise', 'abs'].values
color = np.where(np.array(coefficients['abs']) > noise_coef, 'tab:blue', 'tab:red')

fig, ax = plt.subplots(figsize=(7, 8))
coefficients.plot.barh('feat', 'coef', xerr='std', color=color, ax=ax, legend=False)
plt.axvline(0, color='grey', linestyle='--')
ax.set(title=f"Feature importance, Geographic Weighted Regression model\n", xlabel="Normalized coeficient", ylabel="")
fig.tight_layout()

In [ ]:
coefficients

In [ ]:
## Visualize GWR coefficient values for "nc_agriculture" variable

#Define min and max values
gwr_min = gwr_viz['coef_nc_agriculture'].min()
gwr_max = gwr_viz['coef_nc_agriculture'].max()
vmin = round(np.min(gwr_min), 3)
vmax = round(np.max(gwr_max), 3)

# add axes
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(45,20))
ax.set_title(f'GWR "nc_agriculture" Surface (BW: {str(gwr_bw)}) \n\n Coefficient Range: {str(vmin)} , {str(vmax)}', fontsize=40)


#Set color map
cmap = plt.cm.seismic

#Otherwise, there are positive and negative values so the colormap so zero is the midpoint

cmap = shift_colormap(cmap, start=0.0, midpoint=1 - vmax/(vmax + abs(vmin)), stop=1.)

#Create colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

#Plot GWR parameters
tract2.plot(**{'edgecolor':'black', 'alpha':.4}, ax=ax, color='gray', hatch='\\\\\\\\')
gwr_viz.plot('coef_nc_agriculture', cmap=sm.cmap, ax=ax, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.8})

ax.axis('off')

#Plot gray over insignificant regions
if (gwr_filtered_t[:,0] == 0).any():
    gwr_viz[gwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax, **{'edgecolor':'black'})
    
try:
    fig.tight_layout()    
    fig.subplots_adjust(right=0.9)
    cax = fig.add_axes([0.63, 0.12, 0.04, 0.70])
    cbar = fig.colorbar(sm, cax=cax)
    cbar.ax.tick_params(labelsize=20) 
    cbar.ax.set_xticklabels(ax.get_xticks())
    plt.show()
except ValueError:
    pass

In [ ]:
## plot distribution of coefficients

sns.histplot(gwr_viz['coef_nc_agriculture'], kde=True, color='black')
plt.title('Distribution of "nc_agriculture" Coefficient Values')

In [ ]:
result = (kstest(gwr_viz['coef_nc_agriculture'], cdf='norm'))

print(f"K-S statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
## Separate tracts with positive and negative coefficient values

neg_gardens = gwr_viz[(gwr_viz['coef_nc_agriculture'] < 0)]#gwr_viz['coef_gardens'].quantile(0.25))]
pos_gardens = gwr_viz[(gwr_viz['coef_nc_agriculture'] > 0)]#gwr_viz['coef_gardens'].quantile(0.75))]

In [ ]:
## Take mean values of variables for both sets of tracts

neg_garden_means = social.iloc[:,1:][social['geoid'].isin(neg_gardens['geoid'])].mean()[features]
pos_garden_means = social.iloc[:,1:][social['geoid'].isin(pos_gardens['geoid'])].mean()[features]

In [ ]:
# Combine into single DataFrame

garden_compare = pd.concat([pos_garden_means,neg_garden_means], axis=1)
garden_compare.columns = ['Negative Coef', 'Positive Coef']
garden_compare = garden_compare.drop(index='noise')

In [ ]:
garden_compare

In [ ]:
##visualize mean values in bar chart

ax = garden_compare.div(garden_compare.sum(axis=1), axis=0).plot.bar(figsize=(15,7), color=('dodgerblue','firebrick'))
ax.set_title(f'Mean Variable Values By +/- Sign of "nc_agriculture" Coef \n\n\
Tracts w/ Negative Coefficent: {len(neg_gardens)}\n Tracts w/ Positive Coefficient: {len(pos_gardens)}',color='black')
plt.xticks(rotation=60)
plt.legend(loc=2, prop={'size': 11})

In [ ]:
## Visualize GWR coefficient values for "pluto_church" variable


gwr_min = gwr_viz['coef_pluto_church'].min()
gwr_max = gwr_viz['coef_pluto_church'].max()
vmin = round(np.min(gwr_min), 3)
vmax = round(np.max(gwr_max), 3)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(45,20))
ax.set_title(f'GWR "pluto_church" Surface (BW: {str(gwr_bw)}) \n\n Coefficient Range: {str(vmin)} , {str(vmax)}', fontsize=40)

#Set color map
cmap = plt.cm.seismic

#Otherwise, there are positive and negative values so the colormap so zero is the midpoint

cmap = shift_colormap(cmap, start=0.0, midpoint=1 - vmax/(vmax + abs(vmin)), stop=1.)

#Create colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

#Plot GWR parameters
tract2.plot(**{'edgecolor':'black', 'alpha':.4}, ax=ax, color='gray', hatch='\\\\\\\\')
gwr_viz.plot('coef_pluto_church', cmap=sm.cmap, ax=ax, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.6})
if (gwr_filtered_t[:,0] == 0).any():
    gwr_viz[gwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax, **{'edgecolor':'black'})

ax.axis('off')

try:
    fig.tight_layout()    
    fig.subplots_adjust(right=0.9)
    cax = fig.add_axes([0.63, 0.12, 0.04, 0.70])
    cbar = fig.colorbar(sm, cax=cax)
    cbar.ax.tick_params(labelsize=20) 
    cbar.ax.set_xticklabels(ax.get_xticks())
    plt.show()
except ValueError:
    pass

In [ ]:
sns.histplot(gwr_viz['coef_pluto_church'], kde=True, color='black')
plt.title('Distribution of "pluto_school" Coefficient Values')

In [ ]:
result = (gwr_viz['coef_pluto_church'])

print(f"JB statistic: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
neg_social = gwr_viz[gwr_viz['coef_pluto_church'] < 0]
pos_social = gwr_viz[gwr_viz['coef_pluto_church'] > 0]

neg_social_means = social.iloc[:,1:][social['geoid'].isin(neg_social['geoid'])].mean()[features]
pos_social_means = social.iloc[:,1:][social['geoid'].isin(pos_social['geoid'])].mean()[features]

social_compare = pd.concat([pos_social_means,neg_social_means], axis=1)
social_compare.columns = ['Negative Coef', 'Positive Coef']
social_compare = social_compare.drop(index='noise')

In [ ]:
ax = garden_compare.div(garden_compare.sum(axis=1), axis=0).plot.bar(figsize=(15,7), color=('dodgerblue','firebrick'))
ax.set_title(f'Mean Variable Values By +/- sign of "pluto_church" Coef \n\n\
Tracts w/ Negative Coefficent: {len(neg_gardens)}\n Tracts w/ Positive Coefficient: {len(pos_gardens)}',color='black')
plt.xticks(rotation=60)
plt.show()

In [ ]:
## Visualize GWR coefficient values for "pluto_school" variable

gwr_min = gwr_viz['coef_pluto_school'].min()
gwr_max = gwr_viz['coef_pluto_school'].max()
vmin = round(np.min(gwr_min), 3)
vmax = round(np.max(gwr_max), 3)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(45,20))
ax.set_title(f'GWR "pluto_school" Surface (BW: {str(gwr_bw)}) \n\n Coefficient Range: {str(vmin)} , {str(vmax)}', fontsize=40)

#Set color map
cmap = plt.cm.seismic

#Otherwise, there are positive and negative values so the colormap so zero is the midpoint

cmap = shift_colormap(cmap, start=0.0, midpoint=1 - vmax/(vmax + abs(vmin)), stop=1.)

#Create colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

tract2.plot(**{'edgecolor':'black', 'alpha':.4}, ax=ax, color='gray', hatch='\\\\\\\\')
gwr_viz.plot('coef_pluto_school', cmap=sm.cmap, ax=ax, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.6})
if (gwr_filtered_t[:,0] == 0).any():
    gwr_viz[gwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax, **{'edgecolor':'black'})

ax.axis('off')

try:
    fig.tight_layout()    
    fig.subplots_adjust(right=0.9)
    cax = fig.add_axes([0.63, 0.12, 0.04, 0.70])
    cbar = fig.colorbar(sm, cax=cax)
    cbar.ax.tick_params(labelsize=20) 
    cbar.ax.set_xticklabels(ax.get_xticks())
    plt.show()
except ValueError:
    pass

In [ ]:
sns.histplot(gwr_viz['coef_pluto_school'], kde=True, color='black')
plt.title('Distribution of "pluto_school" Coefficient Values')

In [ ]:
gwr_viz['coef_pluto_school'].quantile(0.25)

In [ ]:
neg_social = gwr_viz[gwr_viz['coef_pluto_school'] < 0]
pos_social = gwr_viz[gwr_viz['coef_pluto_school'] > 0]

neg_social_means = social.iloc[:,1:][social['geoid'].isin(neg_social['geoid'])].mean()[features]
pos_social_means = social.iloc[:,1:][social['geoid'].isin(pos_social['geoid'])].mean()[features]

social_compare = pd.concat([pos_social_means,neg_social_means], axis=1)
social_compare.columns = ['Negative Coef', 'Positive Coef']
social_compare = social_compare.drop(index='noise')

In [ ]:
ax = social_compare.div(social_compare.sum(axis=1), axis=0).plot.bar(figsize=(15,7), color=('dodgerblue','firebrick'))
ax.set_title(f'Mean Variable Values By +/- Sign of "pluto_school" Coef \n\n\
Tracts w/ Negative Coefficent: {len(neg_social)}\n Tracts w/ Positive Coefficient: {len(pos_social)}',color='black')
ax.legend(bbox_to_anchor=(1.0, 1.0))
plt.xticks(rotation=45)
plt.show()

In [ ]:
## Visualize GWR coefficient values for "pluto_vacant" variable

gwr_min = gwr_viz['coef_pluto_vacant'].min()
gwr_max = gwr_viz['coef_pluto_vacant'].max()
vmin = round(np.min(gwr_min), 3)
vmax = round(np.max(gwr_max), 3)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(45,20))
ax.set_title(f'GWR "pluto_vacant" Surface (BW: {str(gwr_bw)}) \n\n Coefficient Range: {str(vmin)} , {str(vmax)}', fontsize=40)

#Set color map
cmap = plt.cm.seismic

#Otherwise, there are positive and negative values so the colormap so zero is the midpoint

cmap = shift_colormap(cmap, start=0.0, midpoint=1 - vmax/(vmax + abs(vmin)), stop=1.)

#Create colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

tract2.plot(**{'edgecolor':'black', 'alpha':.4}, ax=ax, color='gray', hatch='\\\\\\\\')
gwr_viz.plot('coef_pluto_vacant', cmap=sm.cmap, ax=ax, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.6})
if (gwr_filtered_t[:,0] == 0).any():
    gwr_viz[gwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax, **{'edgecolor':'black'})
    
ax.axis('off')

try:
    fig.tight_layout()    
    fig.subplots_adjust(right=0.9)
    cax = fig.add_axes([0.63, 0.12, 0.04, 0.70])
    cbar = fig.colorbar(sm, cax=cax)
    cbar.ax.tick_params(labelsize=20) 
    cbar.ax.set_xticklabels(ax.get_xticks())
    plt.show()
except ValueError:
    pass

In [ ]:
sns.histplot(gwr_viz['coef_pluto_vacant'], kde=True, color='black')
plt.title('Distribution of "pluto_vacant" Coefficient Values')

In [ ]:
neg_social = gwr_viz[gwr_viz['coef_pluto_vacant'] < 0]
pos_social = gwr_viz[gwr_viz['coef_pluto_vacant'] > 0]

In [ ]:
neg_social_means = social.iloc[:,1:][social['geoid'].isin(neg_social['geoid'])].mean()[features]
pos_social_means = social.iloc[:,1:][social['geoid'].isin(pos_social['geoid'])].mean()[features]

In [ ]:
social_compare = pd.concat([pos_social_means,neg_social_means], axis=1)
social_compare.columns = ['Negative Coef', 'Positive Coef']
social_compare = social_compare.drop(index='noise')

In [ ]:
ax = social_compare.div(social_compare.sum(axis=1), axis=0).plot.bar(figsize=(15,7), color=('dodgerblue','firebrick'))
ax.set_title(f'Mean Variable Values By +/- Sign of "pluto_vacant" Coef \n\n\
Tracts w/ Negative Coefficent: {len(neg_social)}\n Tracts w/ Positive Coefficient: {len(pos_social)}',color='black')
ax.legend(bbox_to_anchor=(1.0, 1.0))
plt.xticks(rotation=45)
plt.show()

In [ ]:
## Visualize GWR coefficient values for "pluto_cultural" variable

gwr_min = gwr_viz['coef_pluto_cultural'].min()
gwr_max = gwr_viz['coef_pluto_cultural'].max()
vmin = round(np.min(gwr_min), 3)
vmax = round(np.max(gwr_max), 3)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(45,20))
ax.set_title('GWR "pluto_cultural" Surface (BW: ' + str(gwr_bw) +')'+ 
             '\n\n Coefficient Range = '+ str(vmin) + ' , ' + str(vmax), fontsize=40)

#Set color map
cmap = plt.cm.seismic

#Otherwise, there are positive and negative values so the colormap so zero is the midpoint

cmap = shift_colormap(cmap, start=0.0, midpoint=1 - vmax/(vmax + abs(vmin)), stop=1.)

#Create colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

tract2.plot(**{'edgecolor':'black', 'alpha':.4}, ax=ax, color='gray', hatch='\\\\\\\\')
gwr_viz.plot('coef_pluto_cultural', cmap=sm.cmap, ax=ax, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.6})
if (gwr_filtered_t[:,0] == 0).any():
    gwr_viz[gwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax, **{'edgecolor':'black'})

ax.axis('off')

try:
    fig.tight_layout()    
    fig.subplots_adjust(right=0.9)
    cax = fig.add_axes([0.63, 0.12, 0.04, 0.70])
    cbar = fig.colorbar(sm, cax=cax)
    cbar.ax.tick_params(labelsize=20) 
    cbar.ax.set_xticklabels(ax.get_xticks())
    plt.show()
except ValueError:
    pass

In [ ]:
sns.histplot(gwr_viz['coef_pluto_cultural'], kde=True, color='black')
plt.title('Distribution of "pluto_cultural" Coefficient Values')

In [ ]:
neg_social = gwr_viz[gwr_viz['coef_pluto_cultural'] < 0]
pos_social = gwr_viz[gwr_viz['coef_pluto_cultural'] > 0]

neg_social_means = social.iloc[:,1:][social['geoid'].isin(neg_social['geoid'])].mean()[features]
pos_social_means = social.iloc[:,1:][social['geoid'].isin(pos_social['geoid'])].mean()[features]

social_compare = pd.concat([pos_social_means,neg_social_means], axis=1)
social_compare.columns = ['Negative Coef', 'Positive Coef']
social_compare = social_compare.drop(index='noise')

In [ ]:
neg_social_means = social.iloc[:,1:][social['geoid'].isin(neg_social['geoid'])].mean()[features]
pos_social_means = social.iloc[:,1:][social['geoid'].isin(pos_social['geoid'])].mean()[features]

In [ ]:
social_compare = pd.concat([pos_social_means,neg_social_means], axis=1)
social_compare.columns = ['Negative Coef', 'Positive Coef']
social_compare = social_compare.drop(index='noise')

In [ ]:
ax = social_compare.div(social_compare.sum(axis=1), axis=0).plot.bar(figsize=(15,7), color=('dodgerblue','firebrick'))
ax.set_title(f'Mean Variable Values By +/- Sign of "pluto_cultural" Coef \n\n\
Tracts w/ Negative Coefficent: {len(neg_social)}\n Tracts w/ Positive Coefficient: {len(pos_social)}',color='black')
ax.legend(bbox_to_anchor=(1.0, 1.0))
plt.xticks(rotation=45)
plt.show()

In [ ]:
## Visualize GWR coefficient values for "pluto_outdoor" variable

gwr_min = gwr_viz['coef_pluto_outdoor'].min()
gwr_max = gwr_viz['coef_pluto_outdoor'].max()
vmin = round(np.min(gwr_min), 3)
vmax = round(np.max(gwr_max), 3)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(45,20))
ax.set_title('GWR "pluto_outdoor" Surface (BW: ' + str(gwr_bw) +')'+ 
             '\n\n Coefficient Range = '+ str(vmin) + ' , ' + str(vmax), fontsize=40)

#Set color map
cmap = plt.cm.seismic

#Otherwise, there are positive and negative values so the colormap so zero is the midpoint

cmap = shift_colormap(cmap, start=0.0, midpoint=1 - vmax/(vmax + abs(vmin)), stop=1.)

#Create colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

tract2.plot(**{'edgecolor':'black', 'alpha':.4}, ax=ax, color='gray', hatch='\\\\\\\\')
gwr_viz.plot('coef_pluto_outdoor', cmap=sm.cmap, ax=ax, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.6})
if (gwr_filtered_t[:,0] == 0).any():
    gwr_viz[gwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax, **{'edgecolor':'black'})

ax.axis('off')

try:
    fig.tight_layout()    
    fig.subplots_adjust(right=0.9)
    cax = fig.add_axes([0.63, 0.12, 0.04, 0.70])
    cbar = fig.colorbar(sm, cax=cax)
    cbar.ax.tick_params(labelsize=20) 
    cbar.ax.set_xticklabels(ax.get_xticks())
    plt.axis('off')
    plt.show()
except ValueError:
    pass

In [ ]:
sns.histplot(gwr_viz['coef_pluto_outdoor'], kde=True, color='black')
plt.title('Distribution of "pluto_outdoor" Coefficient Values')

In [ ]:
neg_social = gwr_viz[gwr_viz['coef_pluto_outdoor'] < 0]
pos_social = gwr_viz[gwr_viz['coef_pluto_outdoor'] > 0]

neg_social_means = social.iloc[:,1:][social['geoid'].isin(neg_social['geoid'])].mean()[features]
pos_social_means = social.iloc[:,1:][social['geoid'].isin(pos_social['geoid'])].mean()[features]

social_compare = pd.concat([pos_social_means,neg_social_means], axis=1)
social_compare.columns = ['Negative Coef', 'Positive Coef']
social_compare = social_compare.drop(index='noise')

In [ ]:
ax = social_compare.div(social_compare.sum(axis=1), axis=0).plot.bar(figsize=(15,7), color=('dodgerblue','firebrick'))
ax.set_title(f'Mean Variable Values By +/- Sign of "pluto_outdoor" Coef \n\n\
Tracts w/ Negative Coefficent: {len(neg_social)}\n Tracts w/ Positive Coefficient: {len(pos_social)}',color='black')
ax.legend(bbox_to_anchor=(1.0, 1.0))
plt.xticks(rotation=45)
plt.show()

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(10,8))


sns.histplot(gwr_viz['coef_nc_agriculture'], kde=True, color='black', ax=ax[0,0])
sns.histplot(gwr_viz['coef_pluto_vacant'], kde=True, color='black', ax=ax[0,1])
sns.histplot(gwr_viz['coef_pluto_school'], kde=True, color='black', ax=ax[1,1])
sns.histplot(gwr_viz['coef_pluto_church'], kde=True, color='black', ax=ax[1,0])
sns.histplot(gwr_viz['coef_pluto_cultural'], kde=True, color='black', ax=ax[2,1])
sns.histplot(gwr_viz['coef_pluto_outdoor'], kde=True, color='black', ax=ax[2,0])
plt.tight_layout()

# Lasso Regression

In [ ]:
## Define Target and Explanatory Variables

X = social[features]
y = social['cdc_mental']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=10)

## Standardize values

sc = StandardScaler()
sx = sc.fit(X_train)

X_train_sc = sx.transform(X_train)
X_test_sc = sx.transform(X_test)

## Perform Cross Validation ot find optimal parameter

lasso = LassoCV(n_alphas=1000, random_state=10)
lasso.fit(X_train_sc, y_train)

In [ ]:
predictions = lasso.predict(X_test_sc)
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Out of sample R^2 = {r2:.4f}, MAE = {mae: .4f}, RMSE = {rmse: .4f}')

In [ ]:
## Run model with optimal alpha value over cross-validation

model = Lasso(lasso.alpha_)
cv_model = cross_validate(model, X_train_sc, y_train, cv=RepeatedKFold(), return_estimator=True)

In [ ]:
## Visualize model fit

model.fit(X_train_sc, y_train)

y_pred = model.predict(X_train_sc)
mae = mean_absolute_error(y_train, y_pred)
string_score = f"MAE on training set: {mae:.2f}"

y_pred = model.predict(X_test_sc)
mae = mean_absolute_error(y_test, y_pred)

string_score += f"\nMAE on testing set: {mae:.2f}"

y_pred = model.predict(X_train_sc)
r2 = r2_score(y_train, y_pred)
string_score += f"\n\n R2 on training set: {r2:.2f}"

y_pred = model.predict(X_test_sc)
r2 = r2_score(y_test, y_pred)
string_score += f"\n R2 on testing set: {r2:.2f}"

fig, ax = plt.subplots(figsize=(7, 7))
plt.scatter(y_test, y_pred, alpha=0.5)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c="tab:red")

plt.text(3, 20, string_score)

plt.title(f"Lasso Regression (alpha={lasso.alpha_:.5f}), normalized variables\n")
plt.ylabel("Model predictions")
plt.xlabel("Reported poor mental health")
plt.xlim([0, 27])
plt.ylim([0, 27])
plt.show()

In [ ]:
## Visualize feature coeffeicients

coefficients = {
    'feat': features,
    'coef': lasso.coef_,
    'abs': np.abs(lasso.coef_),
    'std': np.std([est.coef_ for est in cv_model['estimator']], axis=0)
}
coefficients = pd.DataFrame(coefficients).sort_values('abs')
noise_coef = coefficients.loc[coefficients['feat'] == 'noise', 'abs'].values
color = np.where(np.array(coefficients['abs']) > noise_coef, 'tab:blue', 'tab:red')

fig, ax = plt.subplots(figsize=(7, 8))
coefficients.plot.barh('feat', 'coef', xerr='std', color=color, ax=ax, legend=False)
plt.axvline(0, color='grey', linestyle='--')
ax.set(title=f"Feature importance, Lasso model\n", xlabel="Normalized coeficient", ylabel="")
fig.tight_layout()

# Random Forest

In [ ]:
## Find optimal parameters for Random Forest

param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [10, 20, 50],
    'max_features': [.33, .5]
}

forest = RandomForestRegressor(random_state=10)
forest_grid = GridSearchCV(forest, param_grid, cv=5, verbose=1)
forest_grid.fit(X_train, y_train)

In [ ]:
forest_grid.best_params_

In [ ]:
forest_best = forest_grid.best_estimator_
predictions = forest_best.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'MAE = {mae: .4f}, RMSE = {rmse:.4f}')

In [ ]:
## Visualize feature importances

importances = {
    'feat': features,
    'mean': forest_best.feature_importances_,
    'std': np.std([tree.feature_importances_ for tree in forest_best.estimators_], axis=0)
}
importances = pd.DataFrame(importances).sort_values('mean')
noise_mean = importances.loc[importances['feat'] == 'noise', 'mean'].values
color = np.where(np.array(importances['mean']) > noise_mean, 'tab:blue', 'tab:red')

fig, ax = plt.subplots(figsize=(7, 8))
importances.plot.barh('feat', 'mean', xerr='std', color=color, legend=False, ax=ax)
ax.set(title="Feature importances using MDI, Random Forest\n", xlabel="Mean decrease in impurity", ylabel="")
fig.tight_layout()